<a href="https://colab.research.google.com/github/cagBRT/PointCloud/blob/main/point_cloud_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses the Open3d library to learn about point clouds.

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/PointCloud.git cloned-repo
%cd cloned-repo

In [ ]:
from IPython.display import Image

In [ ]:
!pip install open3d

# Import the libaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

# Create a set of points to create a point cloud dataset

In [ ]:
number_points = 8
pcd = np.random.rand(number_points, 3)  # uniform distribution over [0, 1)
print(pcd)

# Plot the point cloud dataset on a 3D plot

In [ ]:
# Create Figure:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter3D(pcd[:, 0], pcd[:, 1], pcd[:, 2])
# label the axes
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Random Point Cloud")
# display:
plt.show()

**Use Plotly to graph the scatterplot**

In [ ]:
import plotly.express as px

fig = px.scatter_3d(pcd, x=pcd[:, 0], y=pcd[:, 1], z=pcd[:, 2],
              )
fig.show()

**Assignment:** <br>
Try different sizes of datasets

# Get a point cloud representation of a bunny

In [ ]:
!wget https://raw.githubusercontent.com/PointCloudLibrary/pcl/master/test/bunny.pcd

In [ ]:
cloud = o3d.io.read_point_cloud("bunny.pcd")
if cloud.is_empty(): exit()

# Estimate normals

In point cloud processing, “estimate normals” refers to the process of computing surface normal vectors for each point in the point cloud.<br>

Surface normals are vectors that are perpendicular to the surface of the object represented by the point cloud at each point.

In [ ]:
Image("/content/cloned-repo/surfaceNormals.png", width=400)

In [ ]:
cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Plot the point cloud

In [ ]:
import plotly.graph_objects as go

In [ ]:
points = np.asarray(cloud.points)

In [ ]:
colors = None
if cloud.has_colors():
    colors = np.asarray(cloud.colors)
elif cloud.has_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(cloud.normals) * 0.5
else:
    geometry.paint_uniform_color((1.0, 0.0, 0.0))
    colors = np.asarray(geometry.colors)

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2],
            mode='markers',
            marker=dict(size=1, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

# Get the mesh version of the bunny

In [ ]:
!wget https://graphics.stanford.edu/~mdfisher/Data/Meshes/bunny.obj

In [ ]:
mesh = o3d.io.read_triangle_mesh("bunny.obj")
if mesh.is_empty(): exit()

Vertex normals are defined at the vertices of the triangle. You can see that they are oriented perpendicular to the smooth underlying surface that the triangle mesh was built from. Sometimes triangles mesh are not directly converted from a smooth surface, and vertex normals have to be computed on the fly.

In [ ]:
Image("interpolated normals.png")

In [ ]:
if not mesh.has_vertex_normals(): mesh.compute_vertex_normals()
if not mesh.has_triangle_normals(): mesh.compute_triangle_normals()

In [ ]:
triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
    colors = tuple(map(tuple, colors))
else:
    colors = (1.0, 0.0, 0.0)

In [ ]:
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            facecolor=colors,
            opacity=0.50)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()

        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)

            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)

    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [ ]:
o3d.visualization.draw_geometries = draw_geometries # replace function
o3d.visualization.draw_geometries([cloud])
o3d.visualization.draw_geometries([mesh])